In [9]:
import gym
import numpy as np

# Define the TSP environment
class TSPEnv(gym.Env):
  # Initialize the environment
  def __init__(self, distances):
    self.distances = distances
    self.num_cities = len(distances)
    self.action_space = gym.spaces.Discrete(self.num_cities)
    self.observation_space = gym.spaces.Discrete(1)

  # Reset the environment
  def reset(self):
    self.route = None
    return 0

  # Step the environment
  def step(self, action):
    if self.route is None:
      self.route = [action]
      return 0, 0, False, {}
    else:
      self.route.append(action)
      if len(self.route) == self.num_cities:
        return 0, self.evaluate(self.route), True, {}
      else:
        return 0, 0, False, {}

  # Render the environment
  def render(self, mode='human'):
    if self.route is None:
      print("Route: []")
    else:
      print("Route:", self.route)

  # Evaluate the fitness of an individual
  def evaluate(self, individual):
    fitness = 0
    for i in range(self.num_cities-1):
      fitness += self.distances[individual[i]][individual[i+1]]
    # Add the distance between the last city and the starting city
    fitness += self.distances[individual[-1]][individual[0]]
    return fitness

# Register the TSP environment
gym.envs.register(id='TSP-v0', entry_point=TSPEnv, kwargs={'distances': distances})

# Create an instance of the TSP environment
env = gym.make('TSP-v0')


c:\Users\fabian.woellenweber\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\envs\registration.py:498: UserWarning: WARN: Overriding environment TSP-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [11]:
import tensorflow as tf

# The distance between each city
distances = [[0, 2, 9, 10],
             [1, 0, 6, 4],
             [8, 9, 0, 7],
             [6, 3, 12, 0]]

# The number of cities
num_cities = len(distances)

# The size of the population
pop_size = 20

# The number of generations
num_generations = 1000

# The learning rate
learning_rate = 0.01

# Create the TSP environment
env = TSPEnv(distances)

# The main function
def solve_tsp():
  # Use TensorFlow to optimize the route
  with tf.compat.v1.Session() as sess:
    # Define the route variable
    route = tf.Variable(tf.random.uniform([env.num_cities], minval=0, maxval=env.num_cities, dtype=tf.int32))

    # Placeholder for the individual
    individual = tf.compat.v1.placeholder(tf.int32, [env.num_cities])

    # One-hot encoding of the individual
    one_hot = tf.compat.v1.one_hot(individual, env.num_cities)

    # The distance between each pair of cities
    distance_matrix = tf.constant(env.distances, dtype=tf.float32)

    # The distance traveled by the individual
    distance = tf.matmul(tf.matmul(one_hot, distance_matrix),
                         tf.transpose(one_hot))

    # The optimization objective is to minimize the distance traveled
    objective = tf.reduce_sum(distance)

    # Use the Adam optimizer to minimize the objective
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(objective)

    # Initialize variables
    sess.run(tf.compat.v1.global_variables_initializer())

    # Iteratively optimize the route
    for generation in range(num_generations):
      # Generate a random population
      population = [env.action_space.sample() for _ in range(pop_size)]

      # Optimize the route for each individual
      for individual in population:
        sess.run(optimizer, feed_dict={individual: individual})
      
      # Calculate the fitness of each individual
      fitnesses = [env.evaluate(x) for x in population]

      # Print the best fitness of the current generation
      print(min(fitnesses))

solve_tsp()


ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'Variable:0' shape=(4,) dtype=int32>"] and loss Tensor("Sum_2:0", shape=(), dtype=float32).

In [13]:
# The main function
def solve_tsp():
  # Use TensorFlow to optimize the route
  with tf.compat.v1.Session() as sess:
    # Define the route variable
    route = tf.Variable(tf.random.uniform([env.num_cities], minval=0, maxval=env.num_cities, dtype=tf.int32))

    # One-hot encoding of the route
    one_hot = tf.one_hot(route, env.num_cities)

    # The distance between each pair of cities
    distance_matrix = tf.constant(env.distances, dtype=tf.float32)

    # The distance traveled by the route
    distance = tf.matmul(tf.matmul(one_hot, distance_matrix),
                         tf.transpose(one_hot))

    # The optimization objective is to minimize the distance traveled
    objective = tf.reduce_sum(distance)

    # Use the Adam optimizer to minimize the objective
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(objective)

    # Initialize variables
    sess.run(tf.compat.v1.global_variables_initializer())

    # Run the optimization for a fixed number of generations
    for i in range(num_generations):
      # Generate a random population
      population = [env.reset() for _ in range(pop_size)]
      # Evaluate the fitness of each individual
      fitnesses = [env.evaluate(individual) for individual in population]
      # Select the best individuals for breeding
      parents = [population[i] for i in np.argsort(fitnesses)[:num_parents]]
      # Crossover and mutation to generate the children
      children = []
      for _ in range(pop_size - num_parents):
        child = crossover(*np.random.choice(parents, 2))
        child = mutate(child, mutation_rate)
        children.append(child)
      # Update the population
      population = parents + children
      # Use TensorFlow to optimize the route
      for individual in population:
        sess.run(optimizer, feed_dict={route: individual})
      # Print the best fitness of the current generation
      print(min(fitnesses))

# Run the TSP solver
solve_tsp()

ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'Variable:0' shape=(4,) dtype=int32>", "<tf.Variable 'Variable_1:0' shape=(4,) dtype=int32>", "<tf.Variable 'Variable_2:0' shape=(4,) dtype=int32>"] and loss Tensor("Sum_4:0", shape=(), dtype=float32).

In [14]:
import gym
import numpy as np

class TSPEnv(gym.Env):
    def __init__(self, cities, distances):
        self.cities = cities
        self.distances = distances
        self.num_cities = len(cities)
        self.action_space = gym.spaces.Discrete(self.num_cities)
        self.observation_space = gym.spaces.Box(0, self.num_cities, shape=(self.num_cities,))
    
    def reset(self):
        self.state = np.zeros(self.num_cities)
        self.state[0] = 1
        self.visited = [0]
        return self.state
    
    def step(self, action):
        self.state[action] = 1
        self.visited.append(action)
        reward = -self.distances[self.visited[-2], action]
        done = len(self.visited) == self.num_cities
        return self.state, reward, done, {}

# Create the TSP environment with a set of cities and distances between them
cities = ['New York', 'Chicago', 'Denver', 'San Francisco', 'Los Angeles']
distances = np.array([[0, 2448, 1434, 1260, 2045],
                      [2448, 0, 2546, 959, 2367],
                      [1434, 2546, 0, 2408, 1745],
                      [1260, 959, 2408, 0, 2295],
                      [2045, 2367, 1745, 2295, 0]])
env = TSPEnv(cities, distances)

# Reset the environment and take an action
state = env.reset()
state, reward, done, _ = env.step(1)

print(state)  # should print [1, 1, 0, 0, 0]
print(reward) # should print -2448
print(done)   # should print False


[1. 1. 0. 0. 0.]
-2448
False


In [15]:
import gym
import numpy as np

# Create the TSP environment with a set of cities and distances between them
cities = ['New York', 'Chicago', 'Denver', 'San Francisco', 'Los Angeles']
distances = np.array([[0, 2448, 1434, 1260, 2045],
                      [2448, 0, 2546, 959, 2367],
                      [1434, 2546, 0, 2408, 1745],
                      [1260, 959, 2408, 0, 2295],
                      [2045, 2367, 1745, 2295, 0]])
env = TSPEnv(cities, distances)

# Initialize the Q-table
num_states = env.observation_space.shape[0]
num_actions = env.action_space.n
Q = np.zeros((num_states, num_actions))

# Define the action selection function
def select_action(state, epsilon=0.1):
    if np.random.rand() < epsilon:
        # Explore: choose a random action
        return np.random.randint(num_actions)
    else:
        # Exploit: choose the action with the highest Q-value
        return np.argmax(Q[state, :])

# Define the Q-learning update function
def update_Q(state, action, reward, next_state, alpha=0.1, gamma=0.9):
    Q[state, action] = (1 - alpha) * Q[state, action] + alpha * (reward + gamma * np.max(Q[next_state, :]))

# Run the TSP loop
num_episodes = 1000
for episode in range(num_episodes):
    # Initialize the environment and state
    state = env.reset()

    while True:
        # Select an action using the action selection function
        action = select_action(state)

        # Take the action and observe the reward and next state
        next_state, reward, done, _ = env.step(action)

        # Update the Q-table using the Q-learning update function
        update_Q(state, action, reward, next_state)

        # Update the state to the next state
        state = next_state

        # If the TSP has been completed, break out of the inner loop
        if done:
            break

# Print the final Q-table
print(Q)

IndexError: arrays used as indices must be of integer (or boolean) type

In [20]:
import gym
import numpy as np

class TSPEnv(gym.Env):
    def __init__(self, distances):
        # distances is a 4x4 matrix of the distances between each pair of cities
        self.distances = distances
        self.state = 0  # current city
        self.done = False  # whether the episode is finished

    def reset(self):
        self.state = 0
        self.done = False
        return self.state

    def step(self, action):
        # action is the index of the city to move to
        if self.done:
            raise ValueError("Episode is finished")
        reward = -self.distances[self.state, action]  # negative of the distance traveled
        self.state = action
        if self.state == 0:  # returned to starting city
            self.done = True
        return self.state, reward, self.done, {}


In [22]:
import gym
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy

# define the distances between the cities
distances = np.array([[0, 10, 15, 20],
                      [10, 0, 35, 25],
                      [15, 35, 0, 30],
                      [20, 25, 30, 0]])

# create the environment
env = TSPEnv(distances)

# define the model
model = Sequential()
model.add(Dense(32, input_dim=1, activation='relu'))  # input is the current city
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='linear'))  # output is a distribution over the actions (cities)

# define the reinforcement learning agent
policy = BoltzmannQPolicy()  # Boltzmann policy with exploration
memory = SequentialMemory(limit=50000, window_length=1)
agent = DQNAgent(model=model, nb_actions=4, memory=memory, policy=policy)

# compile the agent
agent.compile(optimizer='adam', metrics=['mae'])

# fit the agent to the environment
agent.fit(env, nb_steps=10000, visualize=False, verbose=2)

# test the agent
state = env.reset()
for i in range(3):
    action, _, _ = agent.forward(state)
    state, reward, done, _ = env.step(np.argmax(action))
    print(f"State: {state}, Reward: {reward}, Done: {done}")

TypeError: Keras symbolic inputs/outputs do not implement `__len__`. You may be trying to pass Keras symbolic inputs/outputs to a TF API that does not register dispatching, preventing Keras from automatically converting the API call to a lambda layer in the Functional Model. This error will also get raised if you try asserting a symbolic input/output directly.

In [23]:
import gym
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy

# define the distances between the cities
distances = np.array([[0, 10, 15, 20],
                      [10, 0, 35, 25],
                      [15, 35, 0, 30],
                      [20, 25, 30, 0]])

# create the environment
env = TSPEnv(distances)

# define the model
model = Sequential()
model.add(Dense(32, input_dim=1, activation='relu'))  # input is the current city
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='linear'))  # output is a distribution over the actions (cities)

# define the reinforcement learning agent
policy = BoltzmannQPolicy()  # Boltzmann policy with exploration
agent = DQNAgent(model=model, policy=policy)

# compile the agent
agent.compile(optimizer='adam', metrics=['mae'])

# fit the agent to the environment
agent.fit(env, nb_steps=10000, visualize=False, verbose=2)

# test the agent
state = env.reset()
for i in range(3):
    action, _, _ = agent.forward(state)
    state, reward, done, _ = env.step(np.argmax(action))
    print(f"State: {state}, Reward: {reward}, Done: {done}")


TypeError: AbstractDQNAgent.__init__() missing 2 required positional arguments: 'nb_actions' and 'memory'